# Projet maison n° 3

In [1]:
# imports
import pandas as pd
from pathlib import Path
import re

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python `df_names_us()` qui produit un unique DataFrame avec tous les fichiers en utilisant **pandas** (par ex. avec la fonction `pandas.concat()`), pas de bash :)

Ordre et noms des colonnes du DataFrame : "year", "name", "gender", "births"

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini dans les différents fichiers (voir la documentation du dataset).

In [2]:
# names us
def df_names_us():
    files = Path("./names").glob('*.txt')
    df = pd.concat((pd.read_csv(f, names=["name", "gender", "births"])
                    .assign(year=f.stem.split("yob")[1]) for f in files), ignore_index=True, axis=0)
    return df[["year","name","gender","births"]]


df_names_us()

,year,name,gender,births
0,1998,Emily,F,26185
1,1998,Hannah,F,21375
2,1998,Samantha,F,20194
3,1998,Sarah,F,19892
4,1998,Ashley,F,19874
...,...,...,...,...
2052776,1942,Wymon,M,5
2052777,1942,Yancey,M,5
2052778,1942,Yoshio,M,5
2052779,1942,Zach,M,5


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2021_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction `df_names_fr()` qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne "gender"
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : year (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [3]:
# names fr
def df_names_fr():
    mapping = {'2': 'F', '1': 'M'}
    df = pd.read_csv("nat2021.csv", header=0, names=["gender", "name", "year", "births"], sep=";", dtype={"gender":"object"})
    # Suppression des noms de moins de 2 caractères
    df.drop(df[df["name"].str.len() == 1].index, inplace=True)
    # Suppression des _PRENOMS_RARES inutilisables
    df.drop(df[df["name"] == "_PRENOMS_RARES"].index, inplace=True)
    # Suppression des XXXX inutilisables
    df.drop(df[df["year"] == "XXXX"].index, inplace=True)
    # Reset l'index après les suppressions
    df.reset_index(inplace = True)
    # Majuscule à la première lettre des noms
    df["name"] = df["name"].str.title()
    # Conversion en strings des prénoms Na qui sont autrement convertis en "NaN"
    df.loc[df["name"].isna() == True, "name"] = 'Na'
    # Mapping du genre 1 -> M, 2 -> F
    df['gender'] = df['gender'].map(mapping)
    # Tri selon l'ordre souhaité
    df = df.sort_values(by=["year", "gender", "births","name"], ascending=[True,True,False,True], axis=0, ignore_index=True)
    return df[["year", "name", "gender", "births"]]

df_names_fr()

,year,name,gender,births
0,1900,Marie,F,48713
1,1900,Jeanne,F,13981
2,1900,Marguerite,F,8058
3,1900,Germaine,F,6981
4,1900,Louise,F,6696
...,...,...,...,...
648325,2021,Élyam,M,3
648326,2021,Énael,M,3
648327,2021,Éros,M,3
648328,2021,Étan,M,3


## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

Les données sont dans le fichier "Webstat_Export.csv".

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3. On retiendra uniquement les colonnes se terminant effectivement par un code ISO3 entre parenthèses. Par ex., "Dollar des Etats-Unis (USD)".

Implémenter une fonction `df_taux_change()` qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction `pd.to_datetime()` avec le format`"%d/%m/%Y"`) et doit être trié par ordre croissant. Les colonnes du DataFrame doivent correspondre aux devises sélectionnées.

In [4]:
# taux de change
def df_taux_change(devises):
    df = pd.read_csv("Webstat_Export.csv", sep=";", skiprows = [1, 2, 3, 4, 5], decimal = ',', na_values = '-')
    # Renommage de la colonne contenant les dates
    df = df.rename(columns={"Titre :" : "Date"})
    # Réindexation avec les dates au format date
    df["Date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y")
    df.set_index("Date", inplace = True)
    # Tri par ordre croissant
    df.sort_index(ascending=True, inplace = True)
    # Renommer les colonnes
    df = df.rename(columns=lambda x: re.sub(r'([^\(])*(?:\()([^\(]*)(?:\))',r'\2',x))
    # Filtre sur les colonnes choisies
    df = pd.DataFrame(df, columns=devises, dtype="float")
    # Supprime les lignes contenant uniquement des NaN
    df.dropna(how = 'all', inplace = True)
    return df

df_taux_change(["USD","CHF","GBP"])

,USD,CHF,GBP
Date,,,
1999-01-04,1.1789,1.6168,0.71110
1999-01-05,1.1790,1.6123,0.71220
1999-01-06,1.1743,1.6116,0.70760
1999-01-07,1.1632,1.6165,0.70585
1999-01-08,1.1659,1.6138,0.70940
...,...,...,...
2022-09-14,0.9990,0.9612,0.86498
2022-09-15,0.9992,0.9572,0.86934
2022-09-16,0.9954,0.9579,0.87400


### Tests

In [5]:
import unittest

class Session3Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 2052781)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 648330)
        # index
        self.assertTrue(isinstance(df.index, pd.RangeIndex))
        # test names
        self.assertTrue(df.loc[df.name.str.contains(r"^[A-Z]+$")].empty)
        self.assertTrue(df.loc[df.name.str.contains(r"-[a-z]+$")].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df.gender=="F"]) + len(df.loc[df.gender=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(["CHF", "GBP", "USD"])
        # colonnes
        self.assertEqual(list(df.columns), ["CHF", "GBP", "USD"])
        # index
        self.assertTrue(isinstance(df.index, pd.DatetimeIndex))
        # index trié
        self.assertEqual(list(df.index.argsort()), list(range(len(df))))
        # types taux
        self.assertTrue((df.dtypes == float).all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [6]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session3Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [7]:
# run tests
run_tests()

test_df_names_fr (__main__.Session3Tests) ... ok
test_df_names_us (__main__.Session3Tests) ... ok
test_df_taux_change (__main__.Session3Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 2.942s

OK
